# 🧪 Codelab: Quantum Fourier Transform (QFT)

| Metadata | Value |
|----------|-------|
| **Algorithm** | Quantum Fourier Transform |
| **Difficulty** | 🟡 Intermediate |
| **Time** | 90-120 minutes |
| **Prerequisites** | Complex numbers, Hadamard gates, Controlled phases |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Implement QFT circuit from scratch using rotations and swaps
2. ✅ Verify QFT by comparing to the DFT matrix
3. ✅ Implement inverse QFT
4. ✅ Visualize state evolution on Bloch sphere
5. ✅ Understand QFT's role in phase estimation

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT as QiskitQFT

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### Discrete Fourier Transform (DFT)

The DFT transforms a vector $\vec{x} = (x_0, x_1, \ldots, x_{N-1})$ to $\vec{\tilde{x}}$:

$$\tilde{x}_k = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} x_j \cdot \omega_N^{jk}$$

where $\omega_N = e^{2\pi i / N}$ is the **primitive N-th root of unity**.

### Quantum Fourier Transform

QFT maps computational basis to Fourier basis:

$$\text{QFT}|j\rangle = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi ijk/N} |k\rangle$$

### Product Formula

For n qubits, the QFT can be written as a tensor product:

$$\text{QFT}|j_1 j_2 \cdots j_n\rangle = \frac{1}{2^{n/2}} \bigotimes_{l=1}^{n} \left(|0\rangle + e^{2\pi i \cdot 0.j_{n-l+1}\cdots j_n}|1\rangle\right)$$

### Circuit Components

| Gate | Matrix | Purpose |
|------|--------|--------|
| H | $\frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$ | Creates superposition |
| $R_k$ | $\begin{pmatrix} 1 & 0 \\ 0 & e^{2\pi i/2^k} \end{pmatrix}$ | Phase rotation |
| SWAP | Swaps two qubits | Reverses bit order |

## Section 3: Basic Implementation - QFT Building Blocks

In [ ]:
def create_r_gate(k: int) -> np.ndarray:
    """
    Create the R_k rotation gate matrix.
    
    R_k = [[1, 0], [0, e^(2πi/2^k)]]
    
    Args:
        k: The rotation index (R_1 = Z, R_2 = S, R_3 = T, etc.)
    
    Returns:
        2x2 numpy array representing R_k
    """
    theta = 2 * np.pi / (2 ** k)
    return np.array([[1, 0], [0, np.exp(1j * theta)]])


# Demonstrate R gates
print("Rotation Gates R_k = diag(1, e^(2πi/2^k))")
print("=" * 50)

for k in range(1, 5):
    r_k = create_r_gate(k)
    angle = 2 * np.pi / (2 ** k)
    print(f"\nR_{k}: angle = 2π/{2**k} = {np.degrees(angle):.1f}°")
    print(f"  Phase factor: e^(i·{angle:.4f}) = {np.exp(1j*angle):.4f}")
    
# Note equivalence to common gates
print("\n" + "=" * 50)
print("Gate Equivalences:")
print("  R_1 = Z gate (180° rotation)")
print("  R_2 = S gate (90° rotation)")
print("  R_3 = T gate (45° rotation)")

In [ ]:
def qft_rotations(circuit: QuantumCircuit, n: int) -> QuantumCircuit:
    """
    Apply QFT rotations to first n qubits.
    Does NOT include final SWAP gates.
    
    Args:
        circuit: Quantum circuit to modify
        n: Number of qubits to apply QFT to
    
    Returns:
        Modified circuit
    """
    if n == 0:
        return circuit
    
    n_idx = n - 1  # Convert to 0-indexed (apply to highest index first)
    
    # Apply Hadamard to the current qubit
    circuit.h(n_idx)
    
    # Apply controlled rotations
    for qubit in range(n_idx):
        # Control: qubit (lower index)
        # Target: n_idx (current qubit)
        # Rotation: R_{n_idx - qubit + 1}
        k = n_idx - qubit + 1
        angle = 2 * np.pi / (2 ** k)
        circuit.cp(angle, qubit, n_idx)
    
    # Recursively apply to remaining qubits
    qft_rotations(circuit, n - 1)
    
    return circuit


# Demonstrate QFT rotations (without swaps)
n = 3
qc = QuantumCircuit(n)
qft_rotations(qc, n)

print("QFT Rotations (without SWAP):")
print(qc.draw())

In [ ]:
def swap_registers(circuit: QuantumCircuit, n: int) -> QuantumCircuit:
    """
    Swap qubits to reverse the order.
    
    QFT outputs qubits in reversed order, so we need to swap.
    
    Args:
        circuit: Quantum circuit to modify
        n: Number of qubits
    
    Returns:
        Modified circuit
    """
    for qubit in range(n // 2):
        circuit.swap(qubit, n - qubit - 1)
    return circuit


def qft(circuit: QuantumCircuit, n: int) -> QuantumCircuit:
    """
    Apply complete n-qubit QFT.
    
    Args:
        circuit: Quantum circuit to modify
        n: Number of qubits for QFT
    
    Returns:
        Modified circuit with QFT applied
    """
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit


# Complete QFT circuit
n = 3
qc_full = QuantumCircuit(n)
qft(qc_full, n)

print("Complete 3-Qubit QFT Circuit:")
print(qc_full.draw())

## Section 4: Intermediate - Verifying QFT Against DFT Matrix

In [ ]:
def create_dft_matrix(n: int) -> np.ndarray:
    """
    Create the N×N DFT matrix where N = 2^n.
    
    F[j,k] = (1/√N) * ω^(jk) where ω = e^(2πi/N)
    
    Args:
        n: Number of qubits (N = 2^n)
    
    Returns:
        N×N DFT matrix
    """
    N = 2 ** n
    omega = np.exp(2j * np.pi / N)
    
    F = np.zeros((N, N), dtype=complex)
    for j in range(N):
        for k in range(N):
            F[j, k] = omega ** (j * k)
    
    return F / np.sqrt(N)


# Create and display DFT matrices
print("DFT Matrices")
print("=" * 50)

# 1-qubit (N=2)
F_2 = create_dft_matrix(1)
print("\n1-qubit DFT (N=2):")
print(np.round(F_2, 3))
print("Note: This is the Hadamard gate!")

# 2-qubit (N=4)
F_4 = create_dft_matrix(2)
print("\n2-qubit DFT (N=4):")
print(np.round(F_4, 3))

In [ ]:
def verify_qft_matrix(n: int) -> dict:
    """
    Verify that our QFT circuit matches the DFT matrix.
    
    Args:
        n: Number of qubits
    
    Returns:
        Dictionary with verification results
    """
    # Create QFT circuit
    qc = QuantumCircuit(n)
    qft(qc, n)
    
    # Get unitary matrix from circuit
    qft_matrix = Operator(qc).data
    
    # Create DFT matrix
    dft_matrix = create_dft_matrix(n)
    
    # Compare (allowing for global phase)
    # Check if matrices are equal up to global phase
    diff = np.abs(qft_matrix) - np.abs(dft_matrix)
    max_diff = np.max(np.abs(diff))
    
    # For exact comparison, check if product with inverse is identity
    product = qft_matrix @ dft_matrix.conj().T
    is_identity = np.allclose(np.abs(product), np.eye(2**n))
    
    return {
        "qft_matrix": qft_matrix,
        "dft_matrix": dft_matrix,
        "max_abs_diff": max_diff,
        "is_match": is_identity
    }


# Verify for different sizes
print("QFT vs DFT Matrix Verification")
print("=" * 50)

for n in [1, 2, 3, 4]:
    result = verify_qft_matrix(n)
    status = "✓" if result["is_match"] else "✗"
    print(f"{status} n={n} ({2**n}×{2**n}): Max diff = {result['max_abs_diff']:.2e}")

# Show detailed comparison for n=2
print("\nDetailed comparison for n=2:")
result = verify_qft_matrix(2)
print("\nOur QFT matrix:")
print(np.round(result["qft_matrix"], 3))
print("\nDFT matrix:")
print(np.round(result["dft_matrix"], 3))

## Section 5: QFT Action on Basis States

In [ ]:
def qft_on_state(n: int, input_state: int) -> Statevector:
    """
    Apply QFT to a computational basis state.
    
    Args:
        n: Number of qubits
        input_state: Integer representing the basis state (0 to 2^n - 1)
    
    Returns:
        Statevector after QFT
    """
    qc = QuantumCircuit(n)
    
    # Prepare input state
    binary = format(input_state, f'0{n}b')
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            qc.x(i)
    
    # Apply QFT
    qft(qc, n)
    
    return Statevector(qc)


# Analyze QFT action on different input states
n = 3
N = 2 ** n

print(f"QFT Action on {n}-qubit Basis States")
print("=" * 60)

for j in range(min(4, N)):  # Show first 4 states
    state = qft_on_state(n, j)
    print(f"\nQFT|{format(j, f'0{n}b')}⟩ = QFT|{j}⟩:")
    
    # Show amplitudes
    amplitudes = state.data
    print("  Amplitudes (magnitude, phase):")
    for k in range(N):
        amp = amplitudes[k]
        mag = np.abs(amp)
        phase = np.angle(amp)
        if mag > 1e-10:
            print(f"    |{k}⟩: {mag:.3f} ∠ {np.degrees(phase):6.1f}°")
    
    # Verify formula: QFT|j⟩ = (1/√N) Σ_k e^(2πijk/N) |k⟩
    expected_phase_k0 = 0  # k=0 always has phase 0
    expected_phase_k1 = 2 * np.pi * j / N
    print(f"  Expected phase for |1⟩: {np.degrees(expected_phase_k1):.1f}° "
          f"(= 2π×{j}/{N})")

In [ ]:
# Visualize QFT output phases on the complex plane
def plot_qft_phases(n: int, input_states: list):
    """
    Plot QFT output amplitudes on the complex plane.
    """
    N = 2 ** n
    fig, axes = plt.subplots(1, len(input_states), figsize=(4*len(input_states), 4))
    if len(input_states) == 1:
        axes = [axes]
    
    for ax, j in zip(axes, input_states):
        state = qft_on_state(n, j)
        amplitudes = state.data
        
        # Plot unit circle
        theta = np.linspace(0, 2*np.pi, 100)
        ax.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3)
        
        # Plot amplitudes
        colors = plt.cm.viridis(np.linspace(0, 1, N))
        for k in range(N):
            amp = amplitudes[k] * np.sqrt(N)  # Scale for visibility
            ax.arrow(0, 0, amp.real, amp.imag, head_width=0.1, 
                    head_length=0.05, fc=colors[k], ec=colors[k])
            ax.annotate(f'|{k}⟩', (amp.real*1.15, amp.imag*1.15), fontsize=8)
        
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        ax.set_aspect('equal')
        ax.axhline(y=0, color='k', linewidth=0.5)
        ax.axvline(x=0, color='k', linewidth=0.5)
        ax.set_title(f'QFT|{j}⟩ phases (n={n})')
        ax.set_xlabel('Real')
        ax.set_ylabel('Imaginary')
    
    plt.tight_layout()
    plt.show()


# Plot for different input states
plot_qft_phases(3, [0, 1, 2, 4])

## Section 5.5: State Evolution Analysis - Phase Encoding

### 5.5.1 The QFT Identity and "Phase Encoding" Rule

The Quantum Fourier Transform has a beautiful mathematical structure:

$$\text{QFT}|j\rangle = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi i jk/N} |k\rangle = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \omega_N^{jk} |k\rangle$$

where $\omega_N = e^{2\pi i/N}$ is the **primitive N-th root of unity**.

**The Product Representation** (key for circuit implementation):

$$\text{QFT}|j\rangle = \bigotimes_{l=1}^{n} \frac{1}{\sqrt{2}}\left(|0\rangle + e^{2\pi i j/2^l}|1\rangle\right)$$

This says each qubit ends up on the **equator of the Bloch sphere**, with a phase determined by the binary fraction representation of j!

**Binary Fraction Notation:**
$$0.j_l j_{l+1} \cdots j_n = \frac{j_l}{2} + \frac{j_{l+1}}{4} + \cdots + \frac{j_n}{2^{n-l+1}}$$

**The Phase Encoding Rule:**
> "QFT transforms computational basis states (north/south poles) into equatorial states with phases encoding the input value."

From L2.4: *"In the computational basis, all basis states point along the North and South pole... In the Fourier basis, the basis states are pointing along the equator, and they are doing rotations along the equator."*

In [ ]:
import numpy as np

def binary_fraction(j: int, n: int, start_bit: int = 1) -> float:
    """
    Calculate the binary fraction 0.j_start j_{start+1} ... j_n
    
    From L2.5: "This binary fraction representation is key to understanding 
    how the QFT circuit builds up the phases bit by bit."
    
    Args:
        j: Integer to convert
        n: Total number of bits
        start_bit: Which bit position to start from (1-indexed from MSB)
    
    Returns:
        Binary fraction value
    """
    binary = format(j, f'0{n}b')
    fraction = 0.0
    for i, bit in enumerate(binary[start_bit-1:]):
        if bit == '1':
            fraction += 1 / (2 ** (i + 1))
    return fraction


def demonstrate_qft_phases(n: int):
    """
    Show the phase structure of QFT outputs using the product representation.
    
    From L2.4: "Each qubit in the output ends up on the equator with a 
    specific phase determined by the binary fraction of the input."
    """
    N = 2 ** n
    omega = np.exp(2j * np.pi / N)
    
    print("QFT Phase Structure: Product Representation")
    print("=" * 70)
    print(f"n = {n} qubits, N = {N}, ω_N = e^(2πi/{N})")
    print()
    
    print("For input |j⟩, each qubit l gets phase e^(2πi × 0.j_l j_{l+1}...j_n)")
    print("-" * 70)
    
    for j in range(min(N, 8)):  # Show first 8 states
        j_binary = format(j, f'0{n}b')
        print(f"\nQFT|{j}⟩ = QFT|{j_binary}⟩:")
        
        # Build the product state
        product_terms = []
        for l in range(1, n + 1):
            frac = binary_fraction(j, n, l)
            phase = 2 * np.pi * frac
            phase_str = f"2π × {frac:.4f}" if frac > 0 else "0"
            
            # Binary fraction representation
            bin_frac_str = "0." + j_binary[l-1:]
            
            product_terms.append(f"|0⟩ + e^(i{phase_str})|1⟩")
            print(f"  Qubit {l}: (|0⟩ + e^(2πi × {bin_frac_str})|1⟩)/√2")
            print(f"           = (|0⟩ + e^(i × {phase:.4f})|1⟩)/√2")
        
        # Verify with matrix form
        from qiskit.quantum_info import Statevector
        from qiskit import QuantumCircuit
        
        qc = QuantumCircuit(n)
        # Prepare |j⟩
        for i, bit in enumerate(reversed(j_binary)):
            if bit == '1':
                qc.x(i)
        
        # Apply manual QFT
        for i in range(n):
            qc.h(i)
            for k in range(i + 1, n):
                qc.cp(np.pi / (2 ** (k - i)), k, i)
        
        # Swap qubits for correct ordering
        for i in range(n // 2):
            qc.swap(i, n - 1 - i)
        
        state = Statevector(qc)
        print(f"  → Uniform amplitudes: |amp| = 1/√{N} = {1/np.sqrt(N):.4f}")
    
    return omega


omega = demonstrate_qft_phases(3)

### 5.5.2 State Evolution Through the QFT Circuit

The QFT circuit processes qubits sequentially. For each qubit:
1. **Hadamard gate**: Moves from pole to equator
2. **Controlled-R_k gates**: Accumulate phase contributions from other qubits

| Stage | Operation | State of qubit being processed | Key insight |
|-------|-----------|-------------------------------|-------------|
| **Initial** | — | $\|x_1\rangle$ (0 or 1, at pole) | Computational basis = poles |
| **After H** | $H\|x_1\rangle$ | $\frac{1}{\sqrt{2}}(\|0\rangle + (-1)^{x_1}\|1\rangle)$ | Now on equator |
| **After CR₂** | Controlled by $x_2$ | $\frac{1}{\sqrt{2}}(\|0\rangle + e^{2\pi i \cdot 0.x_1 x_2}\|1\rangle)$ | Phase accumulates |
| **After CR₃** | Controlled by $x_3$ | $\frac{1}{\sqrt{2}}(\|0\rangle + e^{2\pi i \cdot 0.x_1 x_2 x_3}\|1\rangle)$ | More precision |
| **...** | ... | ... | ... |
| **Final** | All CR gates | $\frac{1}{\sqrt{2}}(\|0\rangle + e^{2\pi i \cdot 0.x_1 x_2 \ldots x_n}\|1\rangle)$ | Full binary fraction |

**Circuit Pattern** (from L2.5):
```
x₁ ─[H]─[R₂]─[R₃]─...─[Rₙ]─ → (|0⟩ + e^(2πi·0.x₁x₂...xₙ)|1⟩)/√2
        ┌┴┐  ┌┴┐      ┌┴┐
x₂ ─────│●│──│ │──...─│ │── → (process next)
        └─┘  │●│      │ │
x₃ ──────────└─┘──...─│ │──
                      │●│
xₙ ───────────────────└─┘──
```

In [ ]:
def trace_qft_evolution(n: int, j: int, verbose: bool = True) -> dict:
    """
    Trace the state evolution through a QFT circuit step by step.
    
    From L2.5: "The circuit applies H to each qubit, then controlled rotations
    to build up the binary fraction phase bit by bit."
    
    Args:
        n: Number of qubits
        j: Input state (decimal)
        verbose: Print detailed evolution
        
    Returns:
        Dictionary with state at each stage
    """
    from qiskit import QuantumCircuit
    from qiskit.quantum_info import Statevector
    
    j_binary = format(j, f'0{n}b')
    N = 2 ** n
    stages = {}
    
    if verbose:
        print("QFT Circuit Evolution Trace")
        print("=" * 70)
        print(f"Input: |{j}⟩ = |{j_binary}⟩ (n={n} qubits)")
        print()
    
    # Initialize circuit with input state
    qc = QuantumCircuit(n)
    for i, bit in enumerate(reversed(j_binary)):
        if bit == '1':
            qc.x(i)
    
    stages['initial'] = Statevector(qc).data.copy()
    
    if verbose:
        print("Stage 0 - Initial state:")
        print(f"  |{j_binary}⟩ (computational basis)")
        print()
    
    # Process each qubit
    for target in range(n):
        qubit_label = f"x_{target+1}"
        
        # Apply Hadamard
        qc.h(target)
        
        if verbose:
            print(f"Stage {2*target + 1} - H on qubit {target} ({qubit_label}):")
            state = Statevector(qc)
            # Show the key insight: pole → equator
            print(f"  Qubit {target}: pole → equator")
            
            # Calculate the accumulated phase so far for this qubit
            current_phase_bits = j_binary[n-1-target]
            print(f"  Phase contribution: (-1)^{current_phase_bits} = e^(iπ×{current_phase_bits})")
        
        stages[f'after_H_{target}'] = Statevector(qc).data.copy()
        
        # Apply controlled rotations
        for control in range(target + 1, n):
            k = control - target + 1
            angle = np.pi / (2 ** (k - 1))
            qc.cp(angle, control, target)
            
            if verbose:
                control_bit = j_binary[n - 1 - control]
                print(f"  CR_{k} (control=qubit {control}, x_{control+1}={control_bit}):")
                if control_bit == '1':
                    print(f"    → Adds phase e^(iπ/2^{k-1}) = e^(i×{angle:.4f})")
                else:
                    print(f"    → No phase added (control bit is 0)")
        
        stages[f'after_CRs_{target}'] = Statevector(qc).data.copy()
        
        if verbose:
            # Calculate final phase for this qubit
            phase_bits = j_binary[n-1-target:]
            bin_frac = binary_fraction(j, n, n - target)
            print(f"  Final phase for qubit {target}: e^(2πi × 0.{phase_bits}) = e^(i×{2*np.pi*bin_frac:.4f})")
            print()
    
    # Swap qubits for correct output ordering
    if verbose:
        print("Final Stage - Swap qubits for correct ordering:")
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
        if verbose:
            print(f"  SWAP qubits {i} ↔ {n - 1 - i}")
    
    stages['final'] = Statevector(qc).data.copy()
    
    if verbose:
        print()
        print("Output state amplitudes:")
        final_state = stages['final']
        for k in range(N):
            amp = final_state[k]
            phase = np.angle(amp)
            print(f"  |{k}⟩: amplitude = {amp:.4f}, phase = {phase:.4f} rad = {np.degrees(phase):.1f}°")
    
    return stages


# Trace evolution for |5⟩ on 3 qubits
stages = trace_qft_evolution(3, 5)

### 5.5.3 Why the Controlled Rotations? Phase Accumulation

The controlled-$R_k$ gates are the key to building the binary fraction phases:

$$R_k = \begin{pmatrix} 1 & 0 \\ 0 & e^{2\pi i/2^k} \end{pmatrix}$$

**Controlled-$R_k$ action:**
- If control qubit is $|0\rangle$: do nothing
- If control qubit is $|1\rangle$: add phase $e^{2\pi i/2^k}$ to $|1\rangle$ component

**Building the binary fraction:**

Starting with $|x_1\rangle$ after Hadamard: $\frac{1}{\sqrt{2}}(|0\rangle + (-1)^{x_1}|1\rangle)$

| Operation | Phase on $\|1\rangle$ component | Binary fraction so far |
|-----------|--------------------------------|------------------------|
| After H | $e^{i\pi x_1}$ | $0.x_1$ |
| After CR₂ (control=$x_2$) | $e^{i\pi x_1} \cdot e^{i\pi x_2/2}$ | $0.x_1 x_2$ |
| After CR₃ (control=$x_3$) | $e^{i\pi x_1} \cdot e^{i\pi x_2/2} \cdot e^{i\pi x_3/4}$ | $0.x_1 x_2 x_3$ |
| ... | ... | ... |

Since $e^{i\pi x_1} \cdot e^{i\pi x_2/2} \cdot ... = e^{2\pi i(x_1/2 + x_2/4 + ...)} = e^{2\pi i \cdot 0.x_1 x_2 ...}$

**This is why QFT needs $O(n^2)$ gates** - each qubit needs controlled rotations from all subsequent qubits!

In [ ]:
def explore_phase_accumulation(n: int, j: int):
    """
    Visualize how controlled rotations build up the binary fraction phase.
    
    From L2.5: "Each controlled unitary multiplies the |1⟩ coefficient 
    with an exponential factor, building up the binary fraction."
    """
    j_binary = format(j, f'0{n}b')
    
    print("Phase Accumulation Through Controlled Rotations")
    print("=" * 70)
    print(f"Input: |{j}⟩ = |{j_binary}⟩")
    print()
    
    # For each qubit, show how its phase is built
    for target in range(n):
        qubit_idx = target
        print(f"Qubit {qubit_idx} (processes bits x_{qubit_idx+1} through x_{n}):")
        print("-" * 50)
        
        accumulated_phase = 0.0
        phase_terms = []
        
        # After Hadamard: initial phase from own bit
        own_bit = int(j_binary[n - 1 - target])
        h_phase = np.pi * own_bit
        accumulated_phase += h_phase
        
        print(f"  After H: phase = π × x_{qubit_idx+1} = π × {own_bit} = {h_phase:.4f}")
        if own_bit == 1:
            phase_terms.append(f"π")
        
        # After each controlled rotation
        for control in range(target + 1, n):
            k = control - target + 1  # R_k index
            control_bit = int(j_binary[n - 1 - control])
            
            rotation_angle = np.pi / (2 ** (k - 1))
            added_phase = rotation_angle * control_bit
            accumulated_phase += added_phase
            
            print(f"  After CR_{k} (x_{control+1}={control_bit}): ", end="")
            if control_bit == 1:
                print(f"+π/2^{k-1} = +{rotation_angle:.4f}")
                phase_terms.append(f"π/2^{k-1}")
            else:
                print(f"+0 (control is 0)")
        
        # Convert to binary fraction form
        bin_frac_bits = j_binary[n-1-target:]
        bin_frac_value = binary_fraction(j, n, n - target)
        expected_phase = 2 * np.pi * bin_frac_value
        
        print(f"\n  Total accumulated phase: {accumulated_phase:.4f} rad")
        print(f"  Binary fraction: 0.{bin_frac_bits} = {bin_frac_value:.4f}")
        print(f"  Expected: 2π × {bin_frac_value:.4f} = {expected_phase:.4f} rad")
        print(f"  Match: {'✓' if np.isclose(accumulated_phase, expected_phase) else '✗'}")
        print()
    
    print("Summary: Each qubit's final state is:")
    print("  (|0⟩ + e^(2πi × binary_fraction)|1⟩) / √2")
    print("\nThe product of all qubits gives the QFT output!")


explore_phase_accumulation(3, 5)

### 5.5.4 Interactive: QFT vs Classical DFT Comparison

The QFT is exponentially faster than the classical Fast Fourier Transform (FFT):

| Property | Classical FFT | Quantum QFT |
|----------|---------------|-------------|
| Input size | N complex numbers | n = log₂(N) qubits |
| Gate/Op count | O(N log N) | O(n²) = O(log²N) |
| Output | N complex amplitudes | Quantum state (superposition) |
| Readout | Direct access | Measurement (collapses state) |

**Key insight**: QFT is efficient because it operates on a *superposition* of all basis states simultaneously. However, measuring the output collapses it, so QFT is most useful as a *subroutine* (e.g., in Shor's algorithm, QPE) rather than for classical Fourier analysis.

Run the cell below to compare QFT and DFT outputs:

In [ ]:
def compare_qft_dft(n: int, input_state: int):
    """
    Compare Quantum Fourier Transform with classical Discrete Fourier Transform.
    
    From L2.3: "The discrete Fourier transform transforms this to a new 
    coordinate representation given by the matrix transformation."
    """
    from qiskit import QuantumCircuit
    from qiskit.quantum_info import Statevector
    
    N = 2 ** n
    omega = np.exp(2j * np.pi / N)
    
    print("QFT vs Classical DFT Comparison")
    print("=" * 70)
    print(f"n = {n} qubits, N = {N}, input |{input_state}⟩")
    print()
    
    # Classical DFT of basis vector |j⟩
    # DFT|j⟩ = (1/√N) Σ_k ω^(jk) |k⟩
    print("Classical DFT calculation:")
    print("-" * 50)
    
    dft_output = np.zeros(N, dtype=complex)
    for k in range(N):
        dft_output[k] = (1 / np.sqrt(N)) * (omega ** (input_state * k))
        phase = np.angle(dft_output[k])
        print(f"  DFT[{k}] = (1/√{N}) × ω^({input_state}×{k}) = "
              f"{dft_output[k]:.4f} (phase = {np.degrees(phase):.1f}°)")
    
    print()
    
    # Quantum QFT
    print("Quantum QFT calculation:")
    print("-" * 50)
    
    qc = QuantumCircuit(n)
    # Prepare |input_state⟩
    for i, bit in enumerate(reversed(format(input_state, f'0{n}b'))):
        if bit == '1':
            qc.x(i)
    
    # Apply QFT
    for i in range(n):
        qc.h(i)
        for k in range(i + 1, n):
            qc.cp(np.pi / (2 ** (k - i)), k, i)
    for i in range(n // 2):
        qc.swap(i, n - 1 - i)
    
    qft_state = Statevector(qc)
    
    for k in range(N):
        amp = qft_state.data[k]
        phase = np.angle(amp)
        print(f"  QFT[{k}] = {amp:.4f} (phase = {np.degrees(phase):.1f}°)")
    
    print()
    
    # Verify they match
    print("Verification:")
    print("-" * 50)
    match = np.allclose(dft_output, qft_state.data)
    print(f"  DFT ≈ QFT: {'✓ MATCH' if match else '✗ MISMATCH'}")
    
    if match:
        print("\n  The quantum circuit correctly implements the DFT matrix!")
    
    # Gate count comparison
    print()
    print("Complexity comparison:")
    print("-" * 50)
    qft_gates = n * (n + 1) // 2  # H gates + CR gates
    print(f"  QFT gate count: ~{qft_gates} = O(n²) = O({n}²) = O(log²{N})")
    print(f"  Classical FFT: O(N log N) = O({N} × {np.log2(N):.1f}) = O({int(N * np.log2(N))})")
    print(f"  Exponential speedup in gate count!")
    
    return dft_output, qft_state


dft_out, qft_out = compare_qft_dft(3, 5)

### 5.5.5 Summary: Key Insights for Quantum Fourier Transform

| Concept | Mathematical Form | Physical Meaning |
|---------|------------------|------------------|
| **QFT definition** | $\text{QFT}\|j\rangle = \frac{1}{\sqrt{N}}\sum_k \omega_N^{jk}\|k\rangle$ | Transform to frequency domain |
| **Primitive root** | $\omega_N = e^{2\pi i/N}$ | Fundamental rotation angle |
| **Product form** | $\bigotimes_l \frac{1}{\sqrt{2}}(\|0\rangle + e^{2\pi i \cdot 0.j_l...j_n}\|1\rangle)$ | Each qubit gets a binary fraction phase |
| **H gate role** | Pole → equator | Creates superposition on equator |
| **CR_k gate role** | Adds phase $e^{2\pi i/2^k}$ | Builds binary fraction bit by bit |
| **Gate complexity** | $O(n^2)$ | Exponentially faster than classical $O(N \log N)$ |

**Key Takeaways:**
1. 🎯 **Phase encoding**: QFT encodes the input value into phases, not amplitudes
2. 🔄 **Equatorial states**: All output qubits end up on the Bloch sphere equator
3. 📐 **Binary fractions**: The phase of qubit $l$ is determined by $0.j_l j_{l+1} \cdots j_n$
4. ⚡ **Exponential speedup**: $O(\log^2 N)$ vs $O(N \log N)$ gate operations
5. 🧩 **Building block**: QFT is essential for QPE, Shor's algorithm, and many others

**From L2.4:**
> *"In the computational basis, all basis states point along the North and South pole. In the Fourier basis, the basis states are pointing along the equator, rotating at different frequencies."*

---

## Section 6: Inverse QFT

In [ ]:
def inverse_qft(circuit: QuantumCircuit, n: int) -> QuantumCircuit:
    """
    Apply inverse QFT to first n qubits.
    
    The inverse QFT applies the same gates as QFT but in reverse order
    with conjugated phases.
    
    Args:
        circuit: Quantum circuit to modify
        n: Number of qubits
    
    Returns:
        Modified circuit
    """
    # Create QFT circuit
    qft_circ = QuantumCircuit(n, name='QFT')
    qft(qft_circ, n)
    
    # Get inverse (reverses gates and conjugates phases)
    inv_qft_circ = qft_circ.inverse()
    inv_qft_circ.name = 'QFT†'
    
    # Append to main circuit
    circuit.compose(inv_qft_circ, inplace=True)
    
    return circuit


# Show inverse QFT circuit
n = 3
qc_inv = QuantumCircuit(n)
inverse_qft(qc_inv, n)

print("Inverse QFT Circuit (3 qubits):")
print(qc_inv.decompose().draw())

In [ ]:
# Verify QFT^(-1) * QFT = I
def verify_qft_inverse(n: int):
    """
    Verify that QFT^(-1) * QFT = Identity.
    """
    # Create circuit: QFT followed by inverse QFT
    qc = QuantumCircuit(n)
    qft(qc, n)
    inverse_qft(qc, n)
    
    # Get resulting unitary
    U = Operator(qc).data
    
    # Compare to identity
    identity = np.eye(2**n)
    is_identity = np.allclose(U, identity, atol=1e-10)
    
    return is_identity, U


print("Verifying QFT† · QFT = I")
print("=" * 40)

for n in range(1, 5):
    is_identity, U = verify_qft_inverse(n)
    status = "✓" if is_identity else "✗"
    print(f"{status} n={n}: QFT† · QFT {'==' if is_identity else '!='} Identity")

# Show the resulting matrix for n=2
_, U_2 = verify_qft_inverse(2)
print("\nQFT† · QFT for n=2:")
print(np.round(U_2.real, 3))  # Should be 4x4 identity

In [ ]:
# Test round-trip: input → QFT → QFT^(-1) → same input
def test_qft_roundtrip(n: int, input_state: int):
    """
    Test that QFT^(-1)(QFT(|j⟩)) = |j⟩.
    """
    qc = QuantumCircuit(n, n)
    
    # Prepare input state
    binary = format(input_state, f'0{n}b')
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            qc.x(i)
    
    qc.barrier(label='input')
    
    # Apply QFT
    qft(qc, n)
    qc.barrier(label='QFT')
    
    # Apply inverse QFT
    inverse_qft(qc, n)
    qc.barrier(label='QFT†')
    
    # Measure
    qc.measure(range(n), range(n))
    
    # Run
    simulator = AerSimulator()
    result = simulator.run(qc, shots=1024).result()
    counts = result.get_counts()
    
    return qc, counts


# Test round-trip for |101⟩
n = 3
input_state = 5  # |101⟩

qc, counts = test_qft_roundtrip(n, input_state)

print(f"Round-trip test: |{format(input_state, f'0{n}b')}⟩ → QFT → QFT† → |?⟩")
print("=" * 50)
print(f"Results: {counts}")
print(f"Expected: {format(input_state, f'0{n}b')} with 100% probability")
print("\nCircuit:")
print(qc.draw())

## Section 7: Visualization - Bloch Sphere Evolution

In [ ]:
def visualize_qft_evolution(n: int, input_state: int):
    """
    Visualize state evolution through QFT using Bloch spheres.
    """
    print(f"State Evolution: QFT|{format(input_state, f'0{n}b')}⟩")
    print("=" * 50)
    
    # Step 1: Initial state
    qc_init = QuantumCircuit(n)
    binary = format(input_state, f'0{n}b')
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            qc_init.x(i)
    
    state_init = Statevector(qc_init)
    print(f"\nInitial state |{format(input_state, f'0{n}b')}⟩:")
    print(f"  Amplitudes: {np.round(state_init.data, 3)}")
    
    # Step 2: After Hadamard on first qubit (MSB)
    qc_h1 = qc_init.copy()
    qc_h1.h(n-1)  # Hadamard on MSB
    state_h1 = Statevector(qc_h1)
    print(f"\nAfter H on qubit {n-1}:")
    print(f"  Non-zero amplitudes: {sum(np.abs(state_h1.data) > 1e-10)}")
    
    # Step 3: After full QFT
    qc_qft = qc_init.copy()
    qft(qc_qft, n)
    state_qft = Statevector(qc_qft)
    print(f"\nAfter complete QFT:")
    print(f"  All amplitudes have magnitude 1/√{2**n} = {1/np.sqrt(2**n):.4f}")
    print(f"  Phases carry the frequency information")
    
    # Plot Bloch spheres if n <= 3
    if n <= 3:
        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        
        # Initial state
        axes[0].set_title(f'Initial: |{format(input_state, f"{n}b")}⟩')
        
        # Final state  
        axes[1].set_title(f'After QFT')
        
        plt.tight_layout()
        plt.show()
    
    return state_init, state_qft


# Visualize for different inputs
state_init, state_qft = visualize_qft_evolution(3, 5)

In [ ]:
# Compare QFT on |0⟩ vs |j⟩ (different phase patterns)
def compare_qft_outputs(n: int):
    """
    Compare QFT outputs for different input states.
    """
    N = 2 ** n
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    for j in range(min(8, N)):
        ax = axes[j // 4, j % 4]
        
        state = qft_on_state(n, j)
        amplitudes = state.data
        
        # Plot magnitudes and phases
        mags = np.abs(amplitudes)
        phases = np.angle(amplitudes)
        
        x = np.arange(N)
        width = 0.35
        
        ax.bar(x - width/2, mags, width, label='Magnitude', color='blue', alpha=0.7)
        ax.bar(x + width/2, phases / np.pi, width, label='Phase/π', color='red', alpha=0.7)
        
        ax.set_xlabel('Output state |k⟩')
        ax.set_title(f'QFT|{j}⟩')
        ax.set_xticks(x)
        ax.axhline(y=1/np.sqrt(N), color='g', linestyle='--', alpha=0.5)
        
        if j == 0:
            ax.legend()
    
    plt.suptitle(f'{n}-Qubit QFT: Magnitudes are uniform, Phases encode input', fontsize=14)
    plt.tight_layout()
    plt.show()


compare_qft_outputs(3)

## Section 8: Common Traps Demonstration

In [ ]:
# TRAP 1: Forgetting SWAP gates
print("=" * 60)
print("TRAP 1: Forgetting SWAP gates")
print("=" * 60)

n = 3

# Wrong: No swaps
qc_wrong = QuantumCircuit(n)
qft_rotations(qc_wrong, n)  # Only rotations, no swaps!

# Correct: With swaps
qc_correct = QuantumCircuit(n)
qft(qc_correct, n)

# Compare matrices
U_wrong = Operator(qc_wrong).data
U_correct = Operator(qc_correct).data
DFT = create_dft_matrix(n)

print(f"\nMatrix comparison (first row):")
print(f"  No SWAP:   {np.round(U_wrong[0], 2)}")
print(f"  With SWAP: {np.round(U_correct[0], 2)}")
print(f"  DFT:       {np.round(DFT[0], 2)}")

print("\n⚠️  Without SWAP, the output qubits are in reversed order!")

In [ ]:
# TRAP 2: Wrong rotation angles
print("\n" + "=" * 60)
print("TRAP 2: Wrong rotation angles")
print("=" * 60)

def qft_wrong_angles(circuit, n):
    """QFT with WRONG angles (missing factor of 2)."""
    if n == 0:
        return circuit
    n_idx = n - 1
    circuit.h(n_idx)
    for qubit in range(n_idx):
        k = n_idx - qubit + 1
        # WRONG: Missing factor of 2 in angle
        wrong_angle = np.pi / (2 ** k)  # Should be 2*pi/(2**k)
        circuit.cp(wrong_angle, qubit, n_idx)
    qft_wrong_angles(circuit, n - 1)
    return circuit

n = 2
qc_wrong_angle = QuantumCircuit(n)
qft_wrong_angles(qc_wrong_angle, n)
swap_registers(qc_wrong_angle, n)

U_wrong_angle = Operator(qc_wrong_angle).data
DFT_2 = create_dft_matrix(n)

print(f"\nMatrix with WRONG angles:")
print(np.round(U_wrong_angle, 3))
print(f"\nCorrect DFT matrix:")
print(np.round(DFT_2, 3))

print("\n⚠️  Using π/2^k instead of 2π/2^k gives wrong phases!")

In [ ]:
# TRAP 3: Thinking QFT gives direct Fourier coefficients
print("\n" + "=" * 60)
print("TRAP 3: Expecting to read Fourier coefficients directly")
print("=" * 60)

# Apply QFT to a superposition state
n = 3
qc = QuantumCircuit(n, n)

# Prepare a superposition: |ψ⟩ = (|0⟩ + |4⟩)/√2
qc.h(2)  # Creates (|0⟩ + |4⟩)/√2 on computational basis
qc.barrier(label='|ψ⟩')

# Apply QFT
qft(qc, n)
qc.barrier(label='QFT')

# Measure
qc.measure(range(n), range(n))

# Run many times
simulator = AerSimulator()
result = simulator.run(qc, shots=4096).result()
counts = result.get_counts()

print(f"\nInput: |ψ⟩ = (|000⟩ + |100⟩)/√2")
print(f"After QFT + Measurement: {counts}")
print("\n⚠️  Measurement gives RANDOM outcomes from the QFT distribution!")
print("   We cannot directly read all Fourier coefficients.")
print("   QFT is useful as a SUBROUTINE, not for direct DFT computation.")

## Section 9: QFT with Qiskit Library

In [ ]:
# Compare our implementation with Qiskit's built-in QFT
from qiskit.circuit.library import QFT as QiskitQFT

n = 3

# Our implementation
qc_ours = QuantumCircuit(n)
qft(qc_ours, n)

# Qiskit's implementation
qc_qiskit = QiskitQFT(n)

print("Our QFT Implementation:")
print(qc_ours.draw())

print("\nQiskit's QFT Library:")
print(qc_qiskit.decompose().draw())

# Verify they produce the same unitary
U_ours = Operator(qc_ours).data
U_qiskit = Operator(qc_qiskit).data

are_equal = np.allclose(np.abs(U_ours), np.abs(U_qiskit))
print(f"\nMatrices match: {are_equal}")

In [ ]:
# Using Qiskit's QFT options
print("Qiskit QFT Options")
print("=" * 50)

# Without swaps (approximation mode)
qft_no_swap = QiskitQFT(3, do_swaps=False)
print("\n1. Without SWAP gates (do_swaps=False):")
print(qft_no_swap.decompose().draw())

# Approximate QFT (dropping small rotations)
qft_approx = QiskitQFT(4, approximation_degree=1)
print("\n2. Approximate QFT (approximation_degree=1):")
print(f"   Drops rotations with angle < 2π/2^{4-1+1} = {np.degrees(2*np.pi/16):.1f}°")
print(qft_approx.decompose().draw())

# Inverse QFT
inv_qft = QiskitQFT(3, inverse=True)
print("\n3. Inverse QFT:")
print(inv_qft.decompose().draw())

## Section 10: Exercises

### Exercise 1: Build 2-qubit QFT Manually (Beginner)
Construct the 2-qubit QFT circuit using only H and CP gates. Verify it produces the correct DFT matrix.

In [ ]:
# TODO: Exercise 1
# 1. Create a 2-qubit circuit
# 2. Apply H to qubit 1
# 3. Apply controlled-phase (R_2) from qubit 0 to qubit 1
# 4. Apply H to qubit 0
# 5. Apply SWAP
# 6. Verify using Operator

# Your code here:
# qc = QuantumCircuit(2)
# ...

### Exercise 2: Verify QFT Unitarity (Intermediate)
Prove numerically that the QFT matrix satisfies $F^\dagger F = I$ for n=4.

In [ ]:
# TODO: Exercise 2
# 1. Create 4-qubit QFT circuit
# 2. Get the unitary matrix
# 3. Compute F† F
# 4. Verify it equals identity

# Your code here:

### Exercise 3: QFT on Superposition (Intermediate)
Apply QFT to the state $|\psi\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ for n=3 (expanded to 3 qubits). Analyze the output distribution.

In [ ]:
# TODO: Exercise 3
# 1. Create |+⟩ on qubit 0 (others in |0⟩)
# 2. Apply 3-qubit QFT
# 3. Get statevector and analyze amplitudes
# 4. Measure many times and plot histogram

# Your code here:

### Exercise 4: Approximate QFT (Advanced)
Implement approximate QFT that drops controlled rotations smaller than a threshold angle. Compare fidelity vs. gate count for different thresholds.

In [ ]:
# TODO: Exercise 4
# 1. Modify qft_rotations to skip small angles
# 2. Test with thresholds: 0, π/8, π/4, π/2
# 3. Compute fidelity: |⟨ψ_exact|ψ_approx⟩|²
# 4. Plot fidelity vs. gate count

# Your code here:

## Section 11: Quick Knowledge Check

**Q1**: Why does the 1-qubit QFT equal the Hadamard gate?

<details>
<summary>Click for answer</summary>

For N=2, the QFT formula becomes:
- QFT|0⟩ = (1/√2)(|0⟩ + |1⟩) = |+⟩
- QFT|1⟩ = (1/√2)(|0⟩ + e^(iπ)|1⟩) = (1/√2)(|0⟩ - |1⟩) = |-⟩

This is exactly what H does! The Hadamard gate is the Fourier transform on a 2-dimensional space.
</details>

**Q2**: Why do we need SWAP gates at the end of QFT?

<details>
<summary>Click for answer</summary>

The QFT circuit processes qubits from most significant to least significant bit, but outputs them in reversed order. Without SWAP gates, qubit 0 would contain information that should be in qubit n-1 and vice versa. The SWAPs correct this bit reversal.
</details>

**Q3**: Why can't we use QFT to compute classical Fourier transforms faster?

<details>
<summary>Click for answer</summary>

Two fundamental problems:
1. **Loading data**: Converting N classical values to quantum amplitudes requires O(N) operations
2. **Reading results**: QFT output is in superposition. Extracting all N coefficients requires ~N measurements

QFT is powerful as a subroutine where we use interference (not direct readout) to extract information, as in Shor's algorithm.
</details>

## Section 12: Summary & Next Steps

### Key Takeaways

1. **QFT = Quantum DFT**: Transforms computational basis to Fourier basis
2. **Circuit structure**: H gates + controlled phase rotations + SWAPs
3. **Complexity**: $O(n^2)$ gates for n qubits (vs $O(N \log N)$ classical FFT)
4. **1-qubit QFT = Hadamard**: The simplest case
5. **Cannot directly read output**: QFT is a subroutine, not standalone transform

### What's Next?

- **Module 7.5**: Quantum Phase Estimation (uses QFT)
- **Module 7.6**: Shor's Algorithm (uses QFT for period finding)
- **Module 5.2**: VQE (some ansätze use QFT-like structures)

### Applications of QFT

| Algorithm | How QFT is Used |
|-----------|----------------|
| Phase Estimation | Extracts eigenvalue phases |
| Shor's Algorithm | Finds period of modular exponentiation |
| HHL Algorithm | Inverts eigenvalues for linear systems |
| Quantum Simulation | Switching between position/momentum bases |